In [1]:
import urllib3
import certifi
import json
from tqdm.notebook import tqdm

from time import sleep

In [2]:
from requests import get

In [3]:
def open_url(url):
#     header = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
#     }
#     header = {
#         "cookie": "WMF-Last-Access=15-May-2021; WMF-Last-Access-Global=15-May-2021; GeoIP=CN:::34.77:113.72:v4; wikidatawikiwmE-sessionTickLastTickTime=1621085606340; wikidatawikiwmE-sessionTickTickCount=20; wikidatawikiel-sessionId=5a8c5eaa5cefaadc2b6c",
#         "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
#     }
#     http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
#     response = http.request('GET', url, None, header)
#     return response.data.decode('utf-8')
    data = get(url)
    data.encoding="utf-8"
    return data.text

In [4]:
def log_json(js):
    print(json.dumps(js,indent='  '))

In [5]:
def wikiSearch(item_name):
    url = "https://www.wikidata.org/w/api.php?action=wbsearchentities&search=%s&language=en&limit=1&format=json"%(item_name)
    data = json.loads(open_url(url))
    return data["search"][0] if len(data["search"]) > 0 else None

In [6]:
# get("https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q15026&format=json&languages=en").text

In [7]:
def getEntityById(qid):
    url = "https://www.wikidata.org/w/api.php?action=wbgetentities&ids=%s&format=json&languages=en"%(qid)
#     print(url)
    data = json.loads(open_url(url))
    return data['entities'][qid]

In [8]:
def getPropertyById(pid):
    url = "https://www.wikidata.org/wiki/Property:%s"%(pid)
    data = json.loads(open_url(url))
    return data

In [9]:
save_properties = {"P186": "made_from_material", "P279": "subclass_of", "P1552": "has_quality"}
# description
# alias

In [23]:
def get_item_from_wiki(item_name = "mug"):
    found_item = wikiSearch(item_name)
    # log_json(found_item)
    sleep(0.98)
    item_entity = getEntityById(found_item["id"])
    log_json(item_entity)

    export_relationships = {}
    export_relationships["wiki_label"]  = item_entity["labels"]["en"]["value"]
    export_relationships["wiki_id"]     = item_entity["id"]
    export_relationships["description"] = item_entity["descriptions"]["en"]["value"]
    export_relationships["aliases"]     = [] if "en" not in "aliases" else [i["value"] for i in item_entity["aliases"]["en"]]

    for propert in save_properties:
    #     print(propert)
    #     log_json(item_entity["claims"][propert])
        export_relationships[save_properties[propert]] = {}
        if propert not in item_entity["claims"]:
            continue
        for relationship in item_entity["claims"][propert]:
            tmp_item = relationship["mainsnak"]["datavalue"]["value"]["id"]
            export_relationships[save_properties[propert]][tmp_item] = wikiSearch(tmp_item)["label"]

    log_json(export_relationships)
    return export_relationships

In [24]:
from google_images_download import google_images_download
from os.path import join

In [25]:
response = google_images_download.googleimagesdownload()

In [26]:
def download_img_from_google(img_id,img_name, download_num, show_log=False):
    response.download(arguments={"image_directory":img_id, "keywords":img_name, "limit": download_num, "silent_mode":(not show_log)})

In [27]:
item_from_wiki = get_item_from_wiki("bottle")

ConnectionError: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))

In [22]:
download_img_from_google(item_from_wiki["wiki_id"],
                         item_from_wiki["wiki_label"],
                         10,True
                        )


Item no.: 1 --> Item name = bottle
Evaluating...
Could not open URL. Please check your internet connection and/or ssl settings 
If you are using proxy, make sure your proxy settings is configured correctly


SystemExit: 

{
  "wiki_label": "mug",
  "wiki_id": "Q386215",
  "description": "vessel for hot drinks with a handle",
  "aliases": [],
  "made_from_material": {
    "Q45621": "ceramic"
  },
  "subclass_of": {
    "Q81727": "Cup"
  },
  "has_quality": {}
}


In [53]:
# property_id2name = {}

In [54]:
# for i in tqdm(item_entity["claims"]):
#     if i in property_id2name:
#         continue
#     tmpEntity = getEntityById(i)
#     property_id2name[i] = tmpEntity["labels"]["en"]["value"].replace(' ','_')
#     sleep(2)

In [55]:
# log_json(property_id2name)